In [4]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [5]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [6]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='7um'

In [7]:
design.chips

{'C_chip': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '3mm',
   'size_y': '3mm',
   'size_z': '-280um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}},
 'Q_chip': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '7um',
   'size_x': '2mm',
   'size_y': '2mm',
   'size_z': '280um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [8]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_gap='20um',
                                            pad_width='440um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-160}um",
                                              pos_y=f"{q0y+112}um",
                                              pad_width="160um",
                                              pad_height="50um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [9]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [10]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.64mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [11]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [12]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length

In [13]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [14]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 # subtract coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.807562247348839]


In [15]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','800um']
jogs1[3]=['R','200um']
jogs1[4]=['R','615um']
jogs1[5]=['L','200um']
jogs1[6]=['L','615um']


In [16]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '947um', '100um')
            

04:12PM 08s WARNING [check_lengths]: For path table, component=rr0, key=trace has short segments that could cause issues with fillet. Values in (8-10)  are index(es) in shapely geometry.
04:12PM 08s WARNING [check_lengths]: For path table, component=rr0, key=cut has short segments that could cause issues with fillet. Values in (8-10)  are index(es) in shapely geometry.


# EPR Analysis

In [17]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [18]:
hfss.start()

INFO 04:12PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:12PM [load_ansys_project]: 	Opened Ansys App
INFO 04:12PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:12PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   KPS_Flipchip Bump
INFO 04:12PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 04:12PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:12PM [connect]: 	Connected to project "KPS_Flipchip Bump" and design "Design_q3d" 😀 



True

In [19]:
hfss.activate_ansys_design("flipchip7um_Planardesign_Last", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

04:12PM 28s WARNING [activate_ansys_design]: The design_name=flipchip7um_Planardesign_Last was not in active project.  Designs in active project are: 
['flip6um_BUMP', 'flip8um_V6', 'flip8um_V6_BUMP', 'flip6um_V6', 'Planar_Final3', 'flip9um_V6', 'Planar-9', 'flipchip7um_Planar_again', 'Design_q3d', 'flip7um_BUMP', 'flip7um_V6', 'flip5um_V6', 'Planar_Final', 'Planar_Final2'].  A new design will be added to the project.  
INFO 04:12PM [connect_design]: 	Opened active design
	Design:    flipchip7um_Planardesign_Last [Solution type: Eigenmode]
WARNING 04:12PM [connect_setup]: 	No design setup detected.
WARNING 04:12PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:12PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [20]:
hfss.close()


True

# LOM Analysis

In [21]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [22]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.09',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.09',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [24]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:35PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpp9oe110n.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 1, False
INFO 04:35PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp1kfd5t9_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 1, False
INFO 04:35PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp8817knrr.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 2, False
INFO 04:35PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpw7g5jl6n.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 3, False
INFO 04:35PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp2mnc8rfu.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 4, False
INFO 04:35

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,361.930,-114.98000,-108.4200,-74.39500
pad_bot_Q0,-114.980,149.54000,-33.0740,-0.28243
pad_top_Q0,-108.420,-33.07400,149.7900,-6.91240
quad_coupler_c0,-74.395,-0.28243,-6.9124,81.86000


In [28]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 7.33
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.551987 [GHz]
EC 208.904914 [MHz]
EJ 13.616300 [GHz]
alpha -234.953317 [MHz]
dispersion 0.687642 [KHz]
Lq 11.995161 [nH]
Cq 92.722702 [fF]
T1 1808.736642 [us]

**Coupling Properties**

tCqbus1 -3.313893 [fF]
gbus1_in_MHz -39.724896 [MHz]
χ_bus1 -0.093953 [MHz]
1/T1bus1 87.992325 [Hz]
T1bus1 1808.736642 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.00368,255.15758,13.6163,-291.433543,5.650452,[-86.9981820030888],[-0.7541243808731793],0.754124,86.998182
2,4.769847,230.566599,13.6163,-261.219325,1.997576,[-40.66378916021313],[-0.12562892133342796],0.125629,40.663789
3,4.612933,214.844693,13.6163,-242.123102,0.936263,[-39.548087649156294],[-0.09961840674376563],0.099618,39.548088
4,4.576018,211.23591,13.6163,-237.764083,0.777402,[-39.00204047723709],[-0.0930009554908973],0.093001,39.002040
5,4.557651,209.453064,13.6163,-235.613947,0.707903,[-39.7350662125423],[-0.09459057852608406],0.094591,39.735066
6,4.551987,208.904914,13.6163,-234.953317,0.687642,[-39.724896140534966],[-0.09395323209055348],0.093953,39.724896


In [26]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
